In [2]:
%load_ext autoreload
%autoreload 2
from causalboundingengine.algorithms.tianpearl import TianPearl
from causalboundingengine.algorithms.manski import Manski
from causalboundingengine.scenarios import BinaryConf
from causalboundingengine.scenarios import ContIV
from causalboundingengine.scenarios import BinaryIV
import numpy as np
import logging
logging.basicConfig(level=logging.WARNING)

In [2]:
X = np.array([0, 1, 1])  # Example treatment data
Y = np.array([0, 0, 0])  # Example outcome data


alg = TianPearl()  # Instantiate the Manski algorithm

alg.bound_ATE(X, Y)  # Example usage, replace X and Y with actual data

(np.float64(-1.0), np.float64(1.0))

In [5]:
Z = np.array([0, 1, 1, 1, 1])  # Example instrumental variable data
X = np.array([0, 1, 1, 1, 1])  # Example treatment data
Y = np.array([0, 1, 1, 1, 1])  # Example outcome data



scenario = BinaryConf(X, Y) # Instantiate the scenario with the data
scenario.ATE.causaloptim('D:/Program Files/R-4.5.0')

(np.float64(-1.0), np.float64(1.0))